In [19]:
import joblib
from keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

In [2]:
# Placeholder for models and scalers
models = {}
scalers = {}
encoders = {}

In [6]:
strategies = [-2147483648, 0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]

In [7]:
for strategy in strategies:
    # Load the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    scalers[strategy] = joblib.load(scaler_path)

    model_path = f'models/{strategy}_rf_model.pkl'
    models[strategy] = joblib.load(model_path)
    
    # Load the encoder model
    encoder_path = f'encoders/{strategy}_encoder.h5'
    encoders[strategy] = load_model(encoder_path)

    ##

In [9]:
def predict_utilization_all_strategies(new_data):
    results = {}
    for strategy in strategies:
        scaler = scalers[strategy]
        model = models[strategy]
        encoder = encoders[strategy]

        # Standardize new data
        X_new_scaled = scaler.transform(new_data)

        # encode the data
        X_new_encoded = encoder.predict(X_new_scaled)

        # Predict using the model
        prediction = model.predict(X_new_encoded)

        results[strategy] = prediction
    return results

In [20]:
nestdf = pd.read_csv('./Nest.csv')
partavgdf = pd.read_csv('./Part_Avg.csv')
joined_nest_partsavg_df = pd.merge(nestdf, partavgdf, on='ixJobSummary', how='inner')
joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

joined_nest_partsavg_df.drop('dLengthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dWidthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dTrueAreaRectified_Job', axis=1, inplace=True)


joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)
X = joined_nest_partsavg_df.drop(columns=['CropUtil', 'fStrategies'])


In [25]:
X.iloc[654237]

dPartArea_Job              2120.831600
dLength_Avg                  13.132457
dWidth_Avg                    3.879971
dArea_Avg                    19.251057
cNested_Avg                  17.000000
fExtShape_Avg                 7.000000
dExtArea_Avg                 21.953457
dExtBoundaryDist_Avg          1.265157
dExtContainedDist_Avg        14.416871
dLgIntArea_Avg                2.298814
dLgIntBoundaryDist_Avg        0.267543
dLgIntContainedDist_Avg       1.247286
dLgExtConArea_Avg             0.000000
dLgExtConBoundaryDist         0.000000
dLgExtConContainedDist        0.000000
Name: 654237, dtype: float64

In [31]:
X.iloc[1201187]

dPartArea_Job              792.805100
dLength_Avg                  7.585147
dWidth_Avg                   4.883440
dArea_Avg                   31.954447
cNested_Avg                  4.000000
fExtShape_Avg               10.000000
dExtArea_Avg                36.932747
dExtBoundaryDist_Avg         1.849400
dExtContainedDist_Avg        8.584833
dLgIntArea_Avg               3.774960
dLgIntBoundaryDist_Avg       0.387147
dLgIntContainedDist_Avg      1.432107
dLgExtConArea_Avg            2.988960
dLgExtConBoundaryDist        0.258467
dLgExtConContainedDist       1.167313
Name: 1201187, dtype: float64

In [29]:
X.iloc[345038]

dPartArea_Job              4316.764500
dLength_Avg                  18.221422
dWidth_Avg                   19.914244
dArea_Avg                   239.820244
cNested_Avg                   1.000000
fExtShape_Avg                 0.000000
dExtArea_Avg                243.996872
dExtBoundaryDist_Avg          7.494961
dExtContainedDist_Avg        24.091100
dLgIntArea_Avg                4.176633
dLgIntBoundaryDist_Avg        0.423661
dLgIntContainedDist_Avg       3.188983
dLgExtConArea_Avg             8.213883
dLgExtConBoundaryDist         0.777067
dLgExtConContainedDist        4.030811
Name: 345038, dtype: float64

In [35]:

# New data
provided_data = pd.DataFrame({
    'dPartArea_Job': [2120.831600, 792.805100, 4316.764500],
    'dLength_Avg': [13.132457, 7.585147, 18.221422],
    'dWidth_Avg': [3.879971, 4.883440, 19.914244],
    'dArea_Avg': [19.251057, 31.954447, 239.820244],
    'cNested_Avg': [17.000000, 4.000000, 1.000000],
    'fExtShape_Avg': [7.000000, 10.000000, 0.000000],
    'dExtArea_Avg': [21.953457, 36.932747, 243.996872],
    'dExtBoundaryDist_Avg': [1.265157, 1.849400, 7.494961],
    'dExtContainedDist_Avg': [14.416871, 8.584833, 24.091100],
    'dLgIntArea_Avg': [2.298814, 3.774960, 4.176633],
    'dLgIntBoundaryDist_Avg': [0.267543, 0.387147, 0.423661],
    'dLgIntContainedDist_Avg': [1.247286, 1.432107, 3.188983],
    'dLgExtConArea_Avg': [0.000000, 2.988960, 8.213883],
    'dLgExtConBoundaryDist': [0.000000, 0.258467, 0.777067],
    'dLgExtConContainedDist': [0.000000, 1.167313, 4.030811]
})


In [36]:
# Predict utilization for the provided data using all strategies
pred_util_all_strategies = predict_utilization_all_strategies(provided_data)
# for strategy, pred_util in pred_util_all_strategies.items():
#     print(f'Predicted utilization for {strategy}: {pred_util}')

top_strategies = []
for i in range(len(provided_data)):
    # Get predictions for the current row across all strategies
    row_predictions = {strategy: pred_util[i] for strategy, pred_util in pred_util_all_strategies.items()}
    # Sort strategies by predicted utilization
    sorted_strategies = sorted(row_predictions.items(), key=lambda item: item[1], reverse=True)
    # Get the top 3 strategies
    top_3 = sorted_strategies[:3]
    top_strategies.append(top_3)

# Print the top 3 strategies for each row
for idx, top_3 in enumerate(top_strategies):
    print(f'Row {idx + 1}:')
    for strategy, util in top_3:
        print(f'  Strategy: {strategy}, Predicted Utilization: {util}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Row 1:
  Strategy: 16384, Predicted Utilization: 0.6994755071808476
  Strategy: 128, Predicted Utilization: 0.6879479545113848
  Strategy: 2048, Predicted Utilization: 0.6785828409137011
Row 2:
  Strategy: 4, Predicted Utilization: 0.6564003193746727
  Strategy: 64, Predicted Utilization: 0.6452192235792533
  Strategy: 4096, Predicted Utilization: 0.6